In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE StandaloneDeriving #-}
{-# LANGUAGE DerivingVia #-}
{-# LANGUAGE GADTs #-}
{-# LANGUAGE LambdaCase #-}
import GHC.Generics
import Grisette

# Union and Custom Data Types

## Introduction

In the [previous tutorial](./1_symbolic_type.ipynb), we discussed the basic usage of Grisette with symbolic types.
In this tutorial, we will discuss how to write complex programs and use data abstractions that seamlessly integrate with the Grisette system.

In this tutorial, you will learn how to:
- Utilize the [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) monadic container for representing choices and path conditions
- Understand path merging and its importance for efficient symbolic execution
- Work with user-defined data types in Grisette
- Build a program synthesizer using CounterExample-Guided Inductive Synthesis (CEGIS)

By the end of this tutorial, you will have a solid understanding of how to leverage Grisette's features to build sophisticated solver-aided applications.

Please make sure that you have `z3` (https://github.com/Z3Prover/z3) installed and accessible in `$PATH`.

Note that some inline `code blocks` have links to the documentation. It's possible that they are not rendered in a visible way as a link in jupyter notebooks.

## Union Monadic Container

In the previous tutorial, we discussed some primitive symbolic types, like [`SymBool`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymBool) and [`SymInteger`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymInteger). These types are represented as SMT's symbolic formulas, and we can always use SMT's `ite` (in Grisette, [`symIte`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:symIte)) to do symbolic branching on them. However, we often want to work with those user-defined data types beyond SMT formulas in real life, and we need a way to handle their path conditions.

At the core of Grisette lies the [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) monadic container, which is used to represent choices and path conditions in a program.
You can think of it as a list of values from different branches, where each value is associated with a boolean condition (path condition) when the branch is taken.
[`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) helps us integrate user-defined data types into the Grisette system.

To wrap a value in [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union), with a path condition that is always true, you can use the [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Lib-Control-Monad.html#v:mrgReturn) function:

In [2]:
mrgReturn (1, "a") :: Union (SymInteger, SymBool)
mrgReturn (Left 10) :: Union (Either Integer SymBool)

{(1,a)}

{Left 10}

You may wonder why we use [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Lib-Control-Monad.html#v:mrgReturn) instead of Haskell's [`return`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:return) function in the standard library.

The short answer to this is that this is critical for the performance of symbolic evaluation without path explosion. Grisette merges values from the branches with the help of [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) class, which could not be resolved by [`return`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:return). The rule of thumb is to always check if Grisette provides a `mrg*` counterpart to the standard library function and use them if you do not understand Grisette internals. This topic is elaborated in the [optional section for path merging](#Optional:-Path-Merging) in this tutorial.

To introduce branches and path conditions, you can use the [`mrgIf`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgIf) function:

In [3]:
unionList :: Union [SymInteger]
unionList = mrgIf "cond" (mrgReturn ["a1"]) (mrgReturn ["a2", "a3"])
unionList

{If cond [a1] [a2,a3]}

In this example, `unionList` represent a choice between two lists of symbolic integers, depending on the value of the symbolic condition `cond`. If `cond` is true, the value of `unionList` will be `[a1]`, and if `cond` is false, the value will be `[a2,a3]`.

In [4]:
evalSym False (buildModel ("cond" ::= True, "a1" ::= (1 :: Integer))) unionList
evalSym False (buildModel ("cond" ::= False, "a2" ::= (1 :: Integer))) unionList

{[1]}

{[1,a3]}

Conceptually, a [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) value represents an if-then-else tree, where each branch is associated with a path condition. The structure of the `unionList` can be visualized as follows:

```text
unionList:
            +------+
            | cond |
            +------+
       cond /      \ !cond
           /        \
       +------+ +---------+
       | [a1] | | [a2,a3] |
       +------+ +---------+
```

When working with [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union), you can use the standard monadic operations, such as [`>>=`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:-62--62--61-) (bind), or the `do`-notation syntactic sugar, to chain computations and manipulate the values inside the container. Remember to use [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Lib-Control-Monad.html#v:mrgReturn) instead of [`return`](https://hackage.haskell.org/package/base/docs/Control-Monad.html#v:return). For example:

In [5]:
do l <- unionList
   mrgReturn $ 1 : l

{If cond [1,a1] [1,a2,a3]}

In this example, we use the `do` notation to bind the value of `unionList` to the variable `l`, then in each branch, we prepend `1` to the list.

If you don't understand monads, you may checkout the [optional section for monads and unions](#Optional:-Monads-and-Union) in this tutorial.

## Deriving [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) Instance

We mentioned that Grisette performs path merging to help avoid path explosions.
This path merging is controlled by a type class called [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable).

For example, for the type `[SymInteger]`, the Grisette system will try to merge lists with the same lengths, while for the type `Either Integer SymBool`, left values will be merged if they are the same, while right values will always be merged.

In [6]:
mrgIf "cond" (mrgReturn ["a","b"]) (mrgReturn ["c","d"]) :: Union [SymInteger]
mrgIf "cond" (mrgReturn $ Left 1) (mrgReturn $ Left 2) :: Union (Either Integer SymBool)
mrgIf "cond" (mrgReturn $ Left 1) (mrgReturn $ Left 1) :: Union (Either Integer SymBool)
mrgIf "cond" (mrgReturn $ Left 1) (mrgReturn $ Right "x") :: Union (Either Integer SymBool)
mrgIf "cond" (mrgReturn $ Right "x") (mrgReturn $ Right "y") :: Union (Either Integer SymBool)

{[(ite cond a c),(ite cond b d)]}

{If cond (Left 1) (Left 2)}

{Left 1}

{If cond (Left 1) (Right x)}

{Right (ite cond x y)}

To make our custom types work with Grisette, we can implement the [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) type class.

Manually defining a merging strategy is beyond the scope of this tutorial, but we have provided a default instance for non-GADT data types, which you can access using [`DerivingVia`](https://ghc.gitlab.haskell.org/ghc/doc/users_guide/exts/deriving_via.html) with a [`Generic`](https://hackage.haskell.org/package/base/docs/GHC-Generics.html#t:Generic) instance.

In [7]:
data A = X Int SymBool | Y SymInteger
  deriving (Show, Generic)
  deriving (Mergeable) via (Default A)

mrgIf "cond" (return $ X 1 "a") (mrgIf "cond2" (return $ X 2 "b") (return $ X 1 "c")) :: Union A

{If (|| cond (! cond2)) (X 1 (ite cond a c)) (X 2 b)}

Note that a similar mechanism is provided for most of the type classes (e.g., [`EvalSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:EvalSym)) to make the types fully compatible with Grisette.

In [8]:
deriving via (Default A) instance (EvalSym A)
evalSym False (buildModel ("a" ::= True)) $ X 1 "a"

X 1 true

If you don't want to write so many deriving clauses.
Grisette also provides a template haskell procedure implementing all the related type classes for your type.

The following code will derive [`Generic`](https://hackage.haskell.org/package/base/docs/GHC-Generics.html#t:Generic), [`Show`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Show), [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable), [`EvalSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:EvalSym), [`PPrint`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:PPrint), and so on.

Please checkout the documentation for [`deriveAll`](https://hackage.haskell.org/package/grisette/docs/Grisette-TH.html#v:deriveAll) before using it.

```haskell
{-# LANGUAGE TemplateHaskell #-}

data A = X Int SymBool | Y SymInteger
deriveAll ''A
```

## A Rewriting Rule Synthesizer

### Overview
Let's extend the expression equivalence verifier to build a simple rewriting rule synthesizer. The goal is to find an alternative expression that implements the same functionality as the original expression.

We introduce the concept of *program sketches* and *holes*. A *program sketch* represents a *program space* with *holes* that can be instantiated to generate different expressions. *Holes* serve as placeholders for constants. A synthesizer then searches the program space by searching for an instantiation of the holes.

For example, consider the following program sketch for our problem:

```haskell
(If hole1 (Add x hole2) (Mul x hole3)
```

This program sketch represents a program space that includes, but not limited to, the following programs:

- `x + 1` (with `hole1` to be true and `hole2` to be 1)
- `x + 2` (with `hole1` to be true and `hole2` to be 2)
- `x * 3` (with `hole1` to be false and `hole3` to be 3)

Our synthesis goal can be formulated as the following formula:

$p_1 = \exists~\mathrm{hole}\in \mathrm{consts}(e_\mathrm{sketch})\setminus \mathrm{consts}(e_\mathrm{orig}). \forall~\mathrm{var}\in \mathrm{consts}(e_\mathrm{orig}). \mathrm{eval}(e_\mathrm{orig}) = \mathrm{eval}(e_\mathrm{sketch})$

Note that all symbolic constants already present in the original expression are not treated as holes, and our synthesized expression should have the same semantics as the original expression, regardless of the values assigned to these variables.

If you are using Grisette to build a real-world program synthesizer, you may want to check out our work-in-progress synthesis library at https://github.com/lsrcz/grisette-synth-lib, which provides more efficient encoding and an easier-to-use interface for synthesis.

### The DSL

To represent a program sketch, we need to extend our expression type. We modify the operands of `Add`, `Mul` and `Eq` to `Union Expr` to represent a choice among multiple sub-programs. In this tutorial, we will not use GADTs, as we want to simply derive the necessary instances for the type.

In [9]:
data Expr
  = I SymInteger
  | B SymBool
  | Add (Union Expr) (Union Expr)
  | Mul (Union Expr) (Union Expr)
  | Eq (Union Expr) (Union Expr)
  deriving (Show, Eq, Generic)
  deriving (Mergeable, ExtractSym, EvalSym) via (Default Expr)

Additionally, we introduce a sum type `Value` to represent all possible evaluation results.
The evaluation result can be a symbolic integer, a symbolic boolean, or a special `BadValue` if an ill-typed expression is evaluated.

In [10]:
data Value
  = IValue SymInteger
  | BValue SymBool
  | BadValue
  deriving (Show, Eq, Generic)
  deriving (Mergeable, SymEq) via (Default Value)

The `eval` function is updated to handle `Union Expr` and perform dynamic type checking.
The type of our `eval` function is `Expr -> Union Value`, which means that we interpret an expression, and the result is a choice among the value types.

To evaluate `Union Expr`, we can extract the value out of the union structure with the do-notation, or we can use [`onUnion`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:onUnion) or [`.#`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:.-35-) combinator provided by Grisette to lift the `eval` function.



In [11]:
binOp :: Union Expr -> Union Expr -> ((Value, Value) -> Value) -> Union Value
binOp l r f = do
  -- Use do-notation
  lv <- l
  el <- eval lv
  -- Use onUnion combinator
  er <- onUnion eval r
  mrgReturn $ f (el, er)

eval :: Expr -> Union Value
eval (I i) = mrgReturn $ IValue i
eval (B b) = mrgReturn $ BValue b
eval (Add l r) = binOp l r $ \case
  (IValue il, IValue ir) -> IValue $ il + ir
  _ -> BadValue
eval (Mul l r) = binOp l r $ \case
  (IValue il, IValue ir) -> IValue $ il * ir
  _ -> BadValue
eval (Eq l r) = binOp l r $ \case
  (IValue il, IValue ir) -> BValue $ il .== ir
  (BValue il, BValue ir) -> BValue $ il .== ir
  _ -> BadValue

Now we can define some sketches and evaluate them. Our `eval` function is now capable of simultaneously evaluating the entire program space!

In [12]:
sketch :: Expr
sketch = Add (mrgReturn $ I "a") (mrgReturn $ I "b")
sketch
eval sketch

sketch :: Union Expr
sketch = do
  let a = mrgReturn $ I "a"
  let b = mrgReturn $ I "b"
  mrgIf "c" (mrgReturn $ Add a b) (mrgReturn $ Mul a b)
sketch
eval .# sketch

Add {I a} {I b}

{IValue (+ a b)}

{If c (Add {I a} {I b}) (Mul {I a} {I b})}

{IValue (ite c (+ a b) (* a b))}

### The Synthesizer

Finally, let's write the synthesizer. Recall that in our formulation, we have an exists-forall formula. This cannot be directly transformed into an existential formula and efficiently solved, so we use the [CounterExample-Guided Inductive Synthesis (CEGIS)](https://dl.acm.org/doi/10.1145/1168857.1168907) algorithm to handle it by making multiple solver calls, each of which is an existential formula.

The semantics of [`cegisForAll`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:cegisForAll) is to solve the following formula:

$\exists P.(\exists I. \mathrm{pre}(P, I))\wedge(\forall I.\mathrm{pre}(P, I)\Rightarrow\mathrm{post}(P, I))$

You can view $P$ as the program space and $I$ as the program inputs. In our synthesizer, $P$ represents all the holes, and $I$ represents all the variables existing in the original expression. It tries to find a program in the space such that there exists an input fulfilling the precondition, and any input fulfilling the precondition fulfills the postcondition.

The [`cegisForAll`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:cegisForAll) function takes three arguments. The first is the configuration of the solver. The second argument controls which symbolic constants are in $I$. With the [`ExtractSym`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:ExtractSym) instance, we extract all the symbolic constants in the original expression and use them as the set $I$. The third argument specifies the preconditions and postconditions. Here, our precondition is simply true, so we can omit it and use the convenient function [`cegisPostCond`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:cegisPostCond).

In [13]:
synthesisRewriteTarget :: Expr -> Union Expr -> IO ()
synthesisRewriteTarget expr sketch = do
  let lhs = eval expr
  let rhs = eval .# sketch
  r <- cegisForAll z3 expr $ cegisPostCond $ lhs .== rhs
  case r of
    (_, CEGISSuccess model) -> do
      putStrLn "Successfully synthesized RHS:"
      print $ evalSym False model sketch
    (cex, failure) -> do
      putStrLn $ "Synthesis failed with error: " ++ show failure
      putStrLn $ "Counter example list: " ++ show cex

We can now synthesize an expression. The following example tries to determine whether we can rewrite $2 * x$ as $x + x$ or $x * x$.

In [14]:
x :: Union Expr
x = mrgReturn $ I "x"

lhs :: Expr
lhs = Mul (mrgReturn $ I 2) x

sketch :: Union Expr
sketch =
  mrgIf "c"
    (mrgReturn $ Add x x)
    (mrgReturn $ Mul x x)
synthesisRewriteTarget lhs sketch

Successfully synthesized RHS:
{Add {I x} {I x}}

The next example uses a larger sketch. We want to see whether $(a * b) + (b * c)$ can be rewritten.

The sketch we are using is:

```
(?{a,b,c} ?{+,*} ?{a,b,c}) ?{+,*} ?{a,b,c}
```

The question mark indicates that we are selecting among the choices, which is why we call the operator that performs this selection [`choose`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:choose). Note that we need to provide unique names for different choices with [`choose`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:choose), e.g., "lhs1", "rhs".
You can also get unique identifiers within the [`IO`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:IO) monad with [`uniqueIdentifier`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:uniqueIdentifier)
Or you can checkout the [`Fresh`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Fresh) monad provided by Grisette, where fresh names are generated within a monadic context.

In [15]:
a, b, c :: Union Expr
a = mrgReturn $ I "a"
b = mrgReturn $ I "b"
c = mrgReturn $ I "c"
lhs :: Expr
lhs = Add (mrgReturn $ Mul a b) (mrgReturn $ Mul b c)

sketch :: Union Expr
sketch = do
  let lhs1 = chooseUnion [a, b, c] "lhs1"
  let rhs1 = chooseUnion [a, b, c] "rhs1"
  let rhs = chooseUnion [a, b, c] "rhs"
  let lhs = choose [Add lhs1 rhs1, Mul lhs1 rhs1] "lhs"
  choose [Add lhs rhs, Mul lhs rhs] "sketch"

synthesisRewriteTarget lhs sketch

Successfully synthesized RHS:
{Mul {Add {I a} {I c}} {I b}}

## Optional: Monads and Union

For those not familiar with monads, a monad is a type class that defines two operations: [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) and `bind` ([`>>=`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:-62--62--61-) in Haskell).

In the context of [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union):

[`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) wraps a value in the [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) container. Conceptually, it transforms a single value into a [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) value with a true path condition.

```text
a:
        a

return a:
    +-------+
    |   a   |
    +-------+
```

`bind` allows you to chain operations on [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) values, effectively splitting the execution paths based on the path conditions. It takes a `Union a` value and a function `a -> Union b`, and returns a `Union b` value. It applies the function to each branch of the input [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) value, potentially introducing new branches.

In Haskell, the `do` notation is a syntactic sugar for the `bind` and [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) operations. It allows you to write monadic code in a more imperative style, making it easier to read and understand.

For example, the following code using do notation:

```haskell
do l1 <- mrgIf "cond" (return ["a"]) (return ["b","c"])
   mrgIf "cond2" (return $ "d" : l1) (return $ "e" : l1) :: Union [SymInteger]
```

is a syntactic sugar for

```haskell
bind
  (mrgIf "cond" (return ["a"]) (return ["b","c"]))
  (\l1 -> mrgIf "cond2" (return $ "d" : l1) (return $ "e" : l1))
```

In the `do` notation, the left arrow `<-` is used to bind the result of a monadic computation to a variable, which can be used in subsequent computations. The evaluation of the code example could be understood as follows:

```text
Step 1 (l1):
      +------+
      | cond |
      +------+
       /    \
      /      \
  +-----+ +-------+
  | [a] | | [b,c] |
  +-----+ +-------+
Step 2 (apply function in the leaves):
                    +------+
                    | cond |
                    +------+
                     /    \
               +----+      +-----+
              /                   \
         +-------+             +-------+       
         | cond2 |             | cond2 |       
         +-------+             +-------+       
           /   \                /     \        
          /     \              /       \       
    +-------+ +-------+ +---------+ +---------+
    | [d,a] | | [e,a] | | [d,b,c] | | [e,b,c] |
    +-------+ +-------+ +---------+ +---------+
Step 3 (merge, preview for the next section):
                    +------+
                    | cond |
                    +------+
                     /    \
              +-----+      +-----+
             /                    \
+---------------------+ +-----------------------+   
| [(ite cond2 d e),a] | | [(ite cond2 d e),b,c] |  
+---------------------+ +-----------------------+   
```

In [16]:
do l1 <- mrgIf "cond" (return ["a"]) (return ["b","c"])
   mrgIf "cond2" (return $ "d" : l1) (return $ "e" : l1) :: Union [SymInteger]

{If cond [(ite cond2 d e),a] [(ite cond2 d e),b,c]}

## Optional: Path Merging

Path explosion is a common problem in symbolic execution, where the number of paths grows exponentially with the number of branching conditions. This can lead to poor performance and scalability issues when dealing with complex programs.

To understand the path explosion problem, consider the following example:

```haskell
union = do
  a <- union1
  b <- union2
  c <- union3
  return (a, b, c)

furtherCode = do
  v <- union
  return $ g v
```

In this code, `union1`, `union2`, and `union3` are assumed to be [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) values representing choices, and `furtherCode` applies another function `g` to the result of `union`.

If each of `union1`, `union2`, and `union3` has 2 branches, the total number of paths in `union` will be 2^3 = 8. This means that the function `g` will be executed 8 times, once for each possible combination of choices.

This path explosion problem can quickly become intractable as the number of branches increases exponentially with the depth of the computation.

Grisette addresses this problem through path merging.
Instead of exploring each path separately, Grisette merges the paths and represents the result as a symbolic expression.
This allows for efficient symbolic execution without explicitly enumerating all possible paths.

To enable path merging, you should use the [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgReturn) function instead of the vanilla [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) function.
Grisette will then merge the branches when possible in the result.

In [17]:
{- HLINT ignore "Use <$>" -}
do l <- unionList
   return $ sum l
do l <- unionList
   mrgReturn $ sum l

<If cond a1 (+ a2 a3)>

{(ite cond a1 (+ a2 a3))}

In the example, the sum from the two branches has the [`SymInteger`](https://hackage.haskell.org/package/grisette/docs/Grisette-SymPrim.html#t:SymInteger) type, which could be merged with the SMT `ite` operator.
Using this merged union will not cause further computation to be executed for multiple times.

The [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgReturn) function is the key to merging in Grisette.
If we use the vanilla [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) function or [`fmap`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:fmap) function, the result will instead have two branches and will not be merged.
The angle brackets indicate that the result isn't merged.

In [18]:
sum <$> unionList

<If cond a1 (+ a2 a3)>

Let's examine the type signatures of [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) and [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgReturn).

In [19]:
:t return
:t mrgReturn

return :: forall (m :: * -> *) a. Monad m => a -> m a

mrgReturn :: forall (u :: * -> *) a. (MonadTryMerge u, Mergeable a) => a -> u a

The [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return) function has a simple type signature.
It takes a value of type `a` and wraps it in a monadic context `m`.

On the other hand, [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgReturn) has additional constraints.
It requires the value to be an instance of the [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) type class, and the monad `m` to be an instance of the [`MonadTryMerge`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:MonadTryMerge) type class.

The [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) type class defines the merging strategy for different types. By deriving instances of [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) for your custom types, you can enable path merging and avoid the path explosion problem.

The [`MonadTryMerge`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:MonadTryMerge) constraint is used to control the merging behavior of the monad. Those monads capable of merging will cache the merging strategy when calling [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgReturn). This will be used to merge the results of the whole `do`-block.

It's okay that you don't understand everything here.
The key takeaway is to always use [`mrgReturn`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#v:mrgReturn) instead of [`return`](https://hackage.haskell.org/package/base/docs/Prelude.html#v:return), unless you know what you are doing and want to manually control where to merge the values.
We also provide `mrg*` (or sometimes, named `sym*`) variants for many combinators from GHC's base library, including those working with [`Monad`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Monad), [`Applicative`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Applicative), [`Functor`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Functor), [`Foldable`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Foldable), [`Traversable`](https://hackage.haskell.org/package/base/docs/Prelude.html#t:Traversable) and lists.
You may want to check out the documentation in [`Grisette.Lib.*`](https://hackage.haskell.org/package/grisette/docs/Grisette-Lib-Base.html) modules.

In [20]:
return 1 :: Union Int
mrgReturn 1 :: Union Int

<1>

{1}

## Conclusion

In this tutorial, we explored the core construct of Grisette, the [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) monadic container. We learned how to work with [`Union`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Union) to represent choices and introduce path conditions, and how Grisette merges execution paths to improve efficiency and avoid the path explosion problem.

We also discovered how to derive instances of the [`Mergeable`](https://hackage.haskell.org/package/grisette/docs/Grisette-Core.html#t:Mergeable) typeclass for our custom data types, enabling seamless integration with Grisette's features. By deriving instances using [`Generic`](https://hackage.haskell.org/package/base/docs/GHC-Generics.html#t:Generic) and [`DerivingVia`](https://ghc.gitlab.haskell.org/ghc/doc/users_guide/exts/deriving_via.html), we can quickly make our types compatible with Grisette without manually writing the instances.

Furthermore, we extended our expression equivalence verifier to build a simple rewriting rule synthesizer. We introduced the concept of program sketches and holes, and demonstrated how to use CounterExample-Guided Inductive Synthesis (CEGIS) provided by Grisette to search for a program that satisfies a given specification.

However, there are still some areas where the code can be improved. For example, we had to introduce the `BadValue` constructor to handle ill-typed expressions, which adds boilerplate to our code. Additionally, constructing sketches can be verbose and difficult to reuse. We will explore more advanced features of Grisette that can help us address these issues and simplify our code in future tutorials.

By mastering the concepts introduced in this tutorial and leveraging the power of Grisette, you will be well-equipped to tackle complex problems involving symbolic execution, program synthesis, and verification.